# #32 User Defined Results
<i>User defined results are formulae that can be evaluated using model parameters such as geometric and material attribute values as well as load effects from the analysis allowing new results to be displayed in any LUSAS results processing tools. Contours, Reports, Diagrams etc </i>
***

In [ ]:
from shared.LPI import *
lusas = get_lusas_modeller()

if not lusas.existsDatabase():
    raise Exception("This script will add user defined results to an existing model. A model must therefore be loaded")

db = lusas.getDatabase()

Create a simple example to calculate 2D top/bottom stresses in a concrete section and determine if the section is cracked

In [ ]:
# Create a new results entity to contain the user defined results
results_entity = "Concrete Cracking"
db.addUserDefinedResultsEntity(results_entity, "Thick 3D Beam")

In [ ]:
# Convert the tensile strength into model units
fctm = 3.5 # Mpa

fctm = lusas.getUnitSet("N,mm,t,s,C").convertToUnitSet(db.getModelUnits(), fctm, 0, 1, -2, 0, 0, 0)
print(fctm, f"{db.getModelUnits().getForceShortName()}/{db.getModelUnits().getLengthShortName()}²")

Create the user defined results expressions</br>See Utilities > User Defined Results...

In [ ]:
# Create the user defined results attribute to store all the expressions
user_defined_results = db.createUserDefinedResult("User-defined")
# Top and bottom stresses are added to the existing force/moment entity to access the correct results entity, i,e My or a thick beam
user_defined_results.setUserResultComponent("s_top", "Force/Moment - Thick 3D Beam", "Fx / geometric.A + My / geometric.Szt", "Stress_top")
user_defined_results.setUserResultComponent("s_bot", "Force/Moment - Thick 3D Beam", "Fx / geometric.A + My / geometric.Szb", "Stress_bottom")
# Note geometric refers to the geometric (section) attribute. Szb is the section modulus at the negative z axis and is negative 

# Add results to our user entity
# Concrete tensile strength in model units
user_defined_results.setUserResultComponent("fctm", results_entity, fctm, "Tensile capacity")
# maximum tensile stress in top and bottom fibres
user_defined_results.setUserResultComponent("max_fct", results_entity, "max(s_top, s_bot)", "Max tensile stress")
# Determine if the stress exceeds the tensile limit
user_defined_results.setUserResultComponent("isCracked", results_entity, "if(max_fct > fctm, 1, 0)", "1 = cracked, 0 = not cracked")


Set the contour layer to show the user defined result "isCracked" with a two tone contour plot

In [ ]:
lusas.view().insertContoursLayer()
lusas.view().contours().setResults("Concrete Cracking", "isCracked")
lusas.view().contours().chooseSettings(4)